random state =42
training testing 75/25

# With Description

In [1]:
import pandas as pd
import os
data=pd.read_csv(path+"\\data\\data.csv")

data=pd.concat((data,pd.get_dummies(data['RescuerID'])),axis=1)
data.drop('RescuerID',axis=1,inplace=True)

In [18]:
from sklearn.model_selection import train_test_split
X=data.drop("AdoptionSpeed",axis=1)
y=data['AdoptionSpeed']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [19]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion='entropy', max_depth=None, random_state=0)
bag = BaggingClassifier(base_estimator=tree, n_estimators=1000, 
                        max_samples=0.7, bootstrap=True,
                        max_features=1.0, bootstrap_features=False, 
                        n_jobs=1, random_state=1)

In [20]:
bag.fit(X_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=0.7, n_estimators=1000, n_jobs=1, oob_score=False,
         random_state=1, verbose=0, warm_start=False)

In [22]:
from sklearn.metrics import accuracy_score
y_pred = bag.predict(X_test)
y_train_pred=bag.predict(X_train)
print('Accuracy (forest): %.5f' % accuracy_score(y_train, y_train_pred))
print('Accuracy (forest): %.5f' % accuracy_score(y_test, y_pred))

Accuracy (forest): 0.99991
Accuracy (forest): 0.43452


In [23]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

In [25]:
kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel,random_state=0)
gpc.fit(X_train, y_train)

GaussianProcessClassifier(copy_X_train=True,
             kernel=1**2 * RBF(length_scale=1), max_iter_predict=100,
             multi_class='one_vs_rest', n_jobs=None,
             n_restarts_optimizer=0, optimizer='fmin_l_bfgs_b',
             random_state=0, warm_start=False)

In [27]:
y_pred = gpc.predict(X_test)
y_train_pred=gpc.predict(X_train)
print('Accuracy (gp): %.5f' % accuracy_score(y_train, y_train_pred))
print('Accuracy (gp): %.5f' % accuracy_score(y_test, y_pred))

Accuracy (gp): 0.31928
Accuracy (gp): 0.31448


In [29]:
y_predgp=gpc.predict(X_test)
y_predbag = bag.predict(X_test)

In [31]:
a1 = pd.DataFrame(
    {'gp': y_predgp,
     'bag': y_predbag,
    })
a1=a1.set_index("gp")
a1.to_csv(path+"\\result\\gp_and_bagging.csv")

# Without description

In [2]:
newdata=data[data.columns[:21]]

In [3]:
from sklearn.model_selection import train_test_split
X=newdata.drop("AdoptionSpeed",axis=1)
y=newdata['AdoptionSpeed']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [9]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion='entropy', max_depth=13, random_state=0)
bag = BaggingClassifier(base_estimator=tree, n_estimators=1000, 
                        max_samples=0.7, bootstrap=True,
                        max_features=1.0, bootstrap_features=False, 
                        n_jobs=1, random_state=1)

In [10]:
bag.fit(X_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=13,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=0.7, n_estimators=1000, n_jobs=1, oob_score=False,
         random_state=1, verbose=0, warm_start=False)

In [11]:
from sklearn.metrics import accuracy_score
y_predbag = bag.predict(X_test)
y_train_pred=bag.predict(X_train)
print('Accuracy (forest): %.5f' % accuracy_score(y_train, y_train_pred))
print('Accuracy (forest): %.5f' % accuracy_score(y_test, y_predbag))

Accuracy (forest): 0.82818
Accuracy (forest): 0.39851


In [12]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

In [13]:
kernel = 1.0 * RBF(1.0)
gp = GaussianProcessClassifier(kernel=kernel,random_state=0)
gp.fit(X_train, y_train)

GaussianProcessClassifier(copy_X_train=True,
             kernel=1**2 * RBF(length_scale=1), max_iter_predict=100,
             multi_class='one_vs_rest', n_jobs=None,
             n_restarts_optimizer=0, optimizer='fmin_l_bfgs_b',
             random_state=0, warm_start=False)

In [14]:
y_predgp = gp.predict(X_test)
y_train_pred=gp.predict(X_train)
print('Accuracy (gp): %.5f' % accuracy_score(y_train, y_train_pred))
print('Accuracy (gp): %.5f' % accuracy_score(y_test, y_predgp))

Accuracy (gp): 0.35672
Accuracy (gp): 0.35503


In [15]:
a1 = pd.DataFrame(
    {'gp': y_predgp,
     'bag': y_predbag,
    })
a1=a1.set_index("gp")
a1.to_csv(path+"\\result\\gp_and_bagging_without_description.csv")